In [ ]:
%%bash
pip install -r ../../requirements.txt
pip install captum matplotlib seaborn


### Objective
Extract interpretable signals (feature weights, attributions) and analyze misclassifications.


In [ ]:
%%bash
set -e
python - <<'PY'
import numpy as np
from sentiment_experiment import load_dataset, stratified_split, build_text_union, joint_feature_transformer, model_grids
from pathlib import Path
from sklearn.pipeline import Pipeline

df = load_dataset(Path('../../data/teacher_course.csv'))
train_df, _ = stratified_split(df, test_size=0.2)
text_union = build_text_union()
transformer = joint_feature_transformer(text_union)
clf, _ = list(model_grids().values())[0]

pipeline = Pipeline([('features', transformer), ('clf', clf[0])])
pipeline.fit(train_df[['comments', 'aspect']], train_df['sentiment'])
vocab = pipeline.named_steps['features'].transformers_[0][1].transformer_list[0][1].get_feature_names_out()
coefs = pipeline.named_steps['clf'].coef_
top = np.argsort(coefs, axis=1)[:, -10:]
print('Top word features per class:')
for idx, class_name in enumerate(pipeline.named_steps['clf'].classes_):
    print(class_name, [vocab[i] for i in top[idx]])
PY


For transformer models, load your fine-tuned checkpoint and run Captum Integrated Gradients on sample teacher/course texts to visualize token importances.
